# Objective: To detect hate speech using Transformers

1. Loading and Preprocessing the data

2. Training classifier using pre trained ALBERT and fine-tuning it on the data

3. Validating and quantifying the model performance

4. Deploying the model using WebApp and creating an API using cloud

## Set Up

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Install transformers and PyTorch Lightning libraries

!pip install transformers
!pip install pytorch-lightning
!pip install SentencePiece # Required for AlbertTokenizer

In [ ]:
# Import required libraries
from google.colab import drive

import numpy as np
import pandas as pd
import random
import re
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

from transformers import AlbertModel,AlbertTokenizer,DataCollatorWithPadding
from transformers import TrainingArguments, Trainer

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import nltk
from nltk.corpus import stopwords

In [ ]:
#set seed

def set_seeds(seed=1234):
    """Set seeds for reproducibility."""
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


In [ ]:
SEED = 1234
# Set seeds for reproducibility
set_seeds(seed=SEED)

In [ ]:
# Set device
cuda = True
device = torch.device("cuda" if (
    torch.cuda.is_available() and cuda) else "cpu")
torch.set_default_tensor_type("torch.FloatTensor")
if device.type == "cuda":
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
print (device)

cpu


## Load and Preprocess data

In [ ]:
%cd /content/gdrive/MyDrive/Hate Speech/

/content/gdrive/MyDrive/Hate Speech


In [ ]:
# Read the data
hspeech_df = pd.read_csv("./hate_speech_data.csv")
hspeech_df.head()

,Unnamed: 0,tweet,class
0,0,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,1,""" momma said no pussy cats inside my doghouse """,0
2,2,"""@Addicted2Guys: -SimplyAddictedToGuys http://...",0
3,3,"""@AllAboutManFeet: http://t.co/3gzUpfuMev"" woo...",0
4,4,"""@Allyhaaaaa: Lemmie eat a Oreo &amp; do these...",0


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
STOPWORDS = stopwords.words("english")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# nltk.download("stopwords")
print (STOPWORDS[:5])

['i', 'me', 'my', 'myself', 'we']


In [ ]:
def clean_tweet(text):
  '''
  Cleans the input text
  '''

    #lowercase the tweets and remove trailing & ending space
    text = text.lower().strip()                

    # Removes words followed by @
    text = re.sub("(@[A-Za-z0-9]+)", "", text)

    # Removes words at start of string 
    text = re.sub("([^0-9A-Za-z \t])", "", text)

    # remove non alphanumeric chars 
    text = re.sub("[^A-Za-z0-9]+", " ", text)

    #remove stopwords
    words = [word for word in text.split() if word not in STOPWORDS]
    text = " ".join(words)

    # remove multiple spaces
    text = re.sub(" +", " ", text)

    return text

In [ ]:
hspeech_df["tweet"] = hspeech_df["tweet"].apply(clean_tweet)

In [ ]:
hspeech_df.head()

,Unnamed: 0,tweet,class
0,0,rt woman shouldnt complain cleaning house amp ...,0
1,1,momma said pussy cats inside doghouse,0
2,2,simplyaddictedtoguys httptco1jl4hi8zmf woof wo...,0
3,3,httptco3gzupfumev woof woof hot soles,0
4,4,lemmie eat oreo amp dishes one oreo lol,0


## Split Data

In [ ]:
TRAIN_SIZE = 0.7
VAL_SIZE = 0.15
TEST_SIZE = 0.15

In [ ]:
def train_val_test_split(X, y, train_size):
    """Split dataset into data splits."""
    X_train, X_, y_train, y_ = train_test_split(X, y, train_size=TRAIN_SIZE, stratify=y)
    X_val, X_test, y_val, y_test = train_test_split(X_, y_, train_size=0.5, stratify=y_)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
# Data
X = hspeech_df["tweet"].values
y = hspeech_df["class"].values

In [ ]:
# Create data splits
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X=X, y=y, train_size=TRAIN_SIZE)

print (f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print (f"X_val: {X_val.shape}, y_val: {y_val.shape}")
print (f"X_test: {X_test.shape}, y_test: {y_test.shape}")
print (f"Sample point: {X_train[0]} --> {y_train[0]}")

X_train: (3915,), y_train: (3915,)
X_val: (839,), y_val: (839,)
X_test: (839,), y_test: (839,)
Sample point: perhaps ezra miller first crack toward changing regardless whether identifies queer gay --> 0


## Tokenizer

In [ ]:
# Load pre-trained AlbertTokenizer
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
vocab_size = len(tokenizer)
print (vocab_size)

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

30000


In [ ]:
# Tokenize inputs
train_encodings = tokenizer(X_train.tolist(), return_tensors="pt", padding=True,truncation=True, max_length=64)
X_train_ids = train_encodings["input_ids"]
X_train_masks = train_encodings["attention_mask"]
print (X_train_ids.shape, X_train_masks.shape)
val_encodings = tokenizer(X_val.tolist(), return_tensors="pt", padding=True,truncation=True, max_length=64)
X_val_ids = val_encodings["input_ids"]
X_val_masks = val_encodings["attention_mask"]
print (X_val_ids.shape, X_val_masks.shape)
test_encodings = tokenizer(X_test.tolist(), return_tensors="pt", padding=True,truncation=True, max_length=64)
X_test_ids = test_encodings["input_ids"]
X_test_masks = test_encodings["attention_mask"]
print (X_test_ids.shape, X_test_masks.shape)

torch.Size([3915, 64]) torch.Size([3915, 64])
torch.Size([839, 47]) torch.Size([839, 47])
torch.Size([839, 64]) torch.Size([839, 64])


In [ ]:
# Decode
print (f"{X_train_ids[1]}\n{tokenizer.decode(X_train_ids[1])}")

tensor([    2,  5210,   695,  1269,  8148,   383,   110,   164,  2496,  6427,
        12794, 19037,    18,  1954,  2239,     3,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
[CLS] dated girl lived mack road could get holy trinity sacramento ghettos daily basis[SEP]<pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>


In [ ]:
# Sub-word tokens
print (tokenizer.convert_ids_to_tokens(ids=X_train_ids[1]))

['[CLS]', '▁dated', '▁girl', '▁lived', '▁mack', '▁road', '▁could', '▁get', '▁holy', '▁trinity', '▁sacramento', '▁ghetto', 's', '▁daily', '▁basis', '[SEP]', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


## Datasets and Dataloaders

Create Datasets and DataLoaders to be able to efficiently create batches with our data splits

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        if self.labels.tolist():
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
#train,val and test datasets

train_dataset = Dataset(train_encodings, y_train)
val_dataset = Dataset(val_encodings,y_val)
test_dataset = Dataset(test_encodings,y_test)

In [ ]:
# Create dataloaders (train,val and test dataloaders)
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,generator=torch.Generator(device='cuda'))
val_dataloader = DataLoader(val_dataset, batch_size=batch_size,generator=torch.Generator(device='cuda'))
test_dataloader = DataLoader(test_dataset, batch_size=batch_size,generator=torch.Generator(device='cuda'))

for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}


{'input_ids': torch.Size([32, 64]),
 'token_type_ids': torch.Size([32, 64]),
 'attention_mask': torch.Size([32, 64]),
 'labels': torch.Size([32])}

## Trainer

In [ ]:
#creating a trainer class
class Trainer(object):
    def __init__(self, model, device, loss_fn=None, optimizer=None, scheduler=None):

        # Set params
        self.model = model
        self.device = device
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.scheduler = scheduler

    def train_step(self, dataloader):
        """Train step."""
        # Set model to train mode
        self.model.train()
        loss = 0.0

        # Iterate over train batches
        for i, batch in enumerate(dataloader):

            # Step
            inputs = batch['input_ids'].to(device), batch['attention_mask'].to(device)
            targets = batch['labels'].to(device)
            self.optimizer.zero_grad()  # Reset gradients
            z = self.model(inputs)  # Forward pass
            J = self.loss_fn(z, targets)  # Define loss
            J.backward()  # Backward pass
            self.optimizer.step()  # Update weights

            # Cumulative Metrics
            loss += (J.detach().item() - loss) / (i + 1)

        return loss

    def eval_step(self, dataloader):
        """Validation or test step."""
        # Set model to eval mode
        self.model.eval()
        loss = 0.0
        y_trues, y_probs = [], []

        # Iterate over val batches
        with torch.inference_mode():
            for i, batch in enumerate(dataloader):

                # Step
                inputs = batch['input_ids'].to(device), batch['attention_mask'].to(device)
                y_true = batch['labels'].to(device)
                z = self.model(inputs)  # Forward pass
                J = self.loss_fn(z, y_true).item()

                # Cumulative Metrics
                loss += (J - loss) / (i + 1)

                # # Store outputs
                y_prob = torch.exp(z).cpu().numpy()
                y_probs.extend(y_prob)
                y_trues.extend(y_true.cpu().numpy())

        return loss, np.vstack(y_trues), np.vstack(y_probs)

    def train(self, num_epochs, patience, train_dataloader, val_dataloader):
        best_val_loss = np.inf
        for epoch in range(num_epochs):
            # Steps
            train_loss = self.train_step(dataloader=train_dataloader)
            val_loss,_,_ = self.eval_step(dataloader=val_dataloader)
            self.scheduler.step(val_loss)

            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model = self.model
                _patience = patience  # reset _patience
            else:
                _patience -= 1
            if not _patience:  # 0
                print("Stopping early!")
                break

            # Logging
            print(
                f"Epoch: {epoch+1} | "
                f"train_loss: {train_loss:.5f}, "
                f"val_loss: {val_loss:.5f}, "
                f"lr: {self.optimizer.param_groups[0]['lr']:.2E}, "
                f"_patience: {_patience}"
            )
        return best_model

## Albert transformer

In [ ]:
albert_model = AlbertModel.from_pretrained('albert-base-v2')

Downloading:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.decoder.bias', 'predictions.bias', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.dense.bias', 'predictions.decoder.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#defining the transformer class
class Transformer(nn.Module):
    def __init__(self, transformer, dropout_p, embedding_dim, num_classes):
        super(Transformer, self).__init__()
        self.transformer = transformer
        self.dropout = torch.nn.Dropout(dropout_p)
        self.fc1 = torch.nn.Linear(embedding_dim, num_classes)

    def forward(self, inputs):
        ids, masks = inputs
        seq, pool = self.transformer(input_ids=ids, attention_mask=masks,return_dict=False)
        z = self.dropout(pool)
        z = self.fc1(z)
        z = F.log_softmax(z, dim = 1)
        return z

In [ ]:
# Initialize model
dropout_p = 0.5
num_classes = 2
embedding_dim = albert_model.config.hidden_size

model = Transformer(transformer=albert_model, dropout_p=dropout_p,embedding_dim=embedding_dim, num_classes=num_classes)
model = model.to(device)
print(model.named_parameters)

<bound method Module.named_parameters of Transformer(
  (transformer): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_featu

## Training

**Using Learning Rate Scheduler and Early Stopping to counter overfitting**

In [ ]:
# Arguments
lr = 1e-5
num_epochs = 50
patience = 10

In [ ]:
# Class weights
counts = np.bincount(y_train)
class_weights = {i: 1.0/count for i, count in enumerate(counts)}
print (f"counts: {counts}\nweights: {class_weights}")

counts: [2914 1001]
weights: {0: 0.00034317089910775565, 1: 0.000999000999000999}


In [ ]:
# Define loss
class_weights_tensor = torch.Tensor(np.array(list(class_weights.values())))
loss_fn = nn.NLLLoss(weight=class_weights_tensor)

In [ ]:
# Define optimizer & scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=5)

In [ ]:
# Trainer module
trainer = Trainer(model=model, device=device, loss_fn=loss_fn,optimizer=optimizer, scheduler=scheduler)

In [ ]:
# Train
best_model = trainer.train(num_epochs, patience, train_dataloader, val_dataloader)

Epoch: 1 | train_loss: 0.58075, val_loss: 0.42273, lr: 1.00E-05, _patience: 10
Epoch: 2 | train_loss: 0.29773, val_loss: 0.24612, lr: 1.00E-05, _patience: 10
Epoch: 3 | train_loss: 0.18349, val_loss: 0.22971, lr: 1.00E-05, _patience: 10
Epoch: 4 | train_loss: 0.13688, val_loss: 0.23136, lr: 1.00E-05, _patience: 9
Epoch: 5 | train_loss: 0.10802, val_loss: 0.23946, lr: 1.00E-05, _patience: 8
Epoch: 6 | train_loss: 0.07042, val_loss: 0.28741, lr: 1.00E-05, _patience: 7
Epoch: 7 | train_loss: 0.05212, val_loss: 0.27744, lr: 1.00E-05, _patience: 6
Epoch: 8 | train_loss: 0.04205, val_loss: 0.35596, lr: 1.00E-05, _patience: 5
Epoch: 9 | train_loss: 0.02270, val_loss: 0.32304, lr: 1.00E-06, _patience: 4
Epoch: 10 | train_loss: 0.01080, val_loss: 0.31498, lr: 1.00E-06, _patience: 3
Epoch: 11 | train_loss: 0.00715, val_loss: 0.33010, lr: 1.00E-06, _patience: 2
Epoch: 12 | train_loss: 0.00687, val_loss: 0.33134, lr: 1.00E-06, _patience: 1
Stopping early!


## Evaluation on test set

In [ ]:
import json
from sklearn.metrics import classification_report

In [ ]:
# Get predictions on test data
test_loss, y_true, y_prob = trainer.eval_step(dataloader=test_dataloader)
y_pred = np.argmax(y_prob, axis=1)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95       625
           1       0.89      0.79      0.84       214

    accuracy                           0.92       839
   macro avg       0.91      0.88      0.89       839
weighted avg       0.92      0.92      0.92       839



For hate speech detection, precision is more important than recall. Since we want the model to be absolutely sure about the data points that it predicts to be hate speech.

In other words, we care more about the quality of the model predictions than the quantity of them.

In [ ]:
# Save artifacts
from pathlib import Path

dir = Path("final_pytoch_model")
dir.mkdir(parents=True, exist_ok=True)
torch.save(best_model.state_dict(), Path(dir, "hate_speech_model.pt"))

## Inference

In [ ]:
from pathlib import Path

In [ ]:
class Transformer(nn.Module):
    def __init__(self, transformer, dropout_p, embedding_dim, num_classes):
        super(Transformer, self).__init__()
        self.transformer = transformer
        self.dropout = torch.nn.Dropout(dropout_p)
        self.fc1 = torch.nn.Linear(embedding_dim, num_classes)

    def forward(self, inputs):
        ids, masks = inputs
        seq, pool = self.transformer(input_ids=ids, attention_mask=masks,return_dict=False)
        z = self.dropout(pool)
        z = self.fc1(z)
        z = F.log_softmax(z, dim = 1)
        return z

In [ ]:
# Load artifacts
device = torch.device("cpu")
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
transformer = AlbertModel.from_pretrained('albert-base-v2')
embedding_dim = transformer.config.hidden_size

checkpoint = "./final_pytoch_model/hate_speech_model.pt"
model = Transformer(transformer=transformer, dropout_p=0.5,embedding_dim=embedding_dim, num_classes=2)
model.load_state_dict(torch.load(checkpoint,map_location=device))

Downloading:   0%|          | 0.00/760k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/684 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<All keys matched successfully>

In [ ]:
# text cleaning and generating ids , masks
text = "you're such a retard i hope you get type 2 diabetes and die from a sugar rush you fucking faggot @Dare_ILK"

In [ ]:
#Inference Function
def hatespeech(text):
  X = clean_tweet(text)
  encoded_input = tokenizer(X, return_tensors="pt", padding=True,truncation=True, max_length=512).to(torch.device("cpu"))
  ids = encoded_input["input_ids"]
  masks = encoded_input["attention_mask"]

  # Forward pass w/ inputs
  inputs = ids,masks
  model.eval()
  z = model(inputs)
  # Output probababilites
  y_prob = torch.exp(z).detach().cpu().numpy()[0]
  if np.argmax(y_prob) == 1:
    return 'This tweet/text is a hate speech'
  else:
    return ' This tweet/text is not a hate speech'


In [ ]:
hatespeech(text)

'This tweet/text is a hate speech'